# Day 8: Model Followers

Complicated and unrewarding one for today, with lots of learning required to pull it off for little results. But anyway, here we go. Determines age and gender of Twitter followers according to profile photo, using cutting-edge CNN published this year. For the paper, see [here](http://www.openu.ac.il/home/hassner/projects/cnn_agegender/). Somewhat inspired by [this](https://blog.scraperwiki.com/2014/02/face-rekognition/).

Pipeline:

- Get full-size images of all profiles of twitter followers
    - Straighforward. Use `tweepy` and strip `_original` from returned profile pic
    - Ran into ratelimiting. Better to use the follower_id getter call and resolve to User objects (batches of 100)
- Convert to JPEG
    - `imagemagick`
- Use OpenCV to find faces and crop to bounding box
    - Only picked images where one face found
    - Extensive code borrowing from [here](https://realpython.com/blog/python/face-recognition-with-python/)
- Spin up a docker container with Caffe dependencies installed
    - Worth a blog post in and of itself
    - Getting the images in was fun
- Go crazy realizing demo code does not work out of the box (does it ever?)
- Modify code from CNN site above to output filename, est. age, est. gender
- Realize output sucks

In [22]:
import tweepy
import urllib

In [49]:
CONSUMER_KEY = ''
CONSUMER_SECRET = ''
ACCESS_TOKEN_KEY= ''
ACCESS_TOKEN_SECRET = ''

In [7]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN_KEY, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

In [ ]:
api.followers_ids

In [12]:
followers = []

In [13]:
for follower in tweepy.Cursor(api.followers).items():
    followers.append(follower)

TweepError: [{u'message': u'Rate limit exceeded', u'code': 88}]

In [16]:
len(followers)

260

In [ ]:
urllib.urlretrieve()

In [23]:
for follower in followers:
    urllib.urlretrieve(follower.profile_image_url_https.replace('_normal', ''), follower.screen_name)

In [29]:
import cv2
import sys
import glob

for imagePath in glob.glob('profiles/*'):
    # Get user supplied values
    cascPath = 'FaceDetect/haarcascade_frontalface_default.xml'

    # Create the haar cascade
    faceCascade = cv2.CascadeClassifier(cascPath)

    # Read the image
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags = cv2.cv.CV_HAAR_SCALE_IMAGE
    )

    if len(faces) == 1:
        face = faces[0]
        x, y, w, h = face
        cropped = image[y:y+h, x:x+w]
        cv2.imwrite(imagePath.replace('.jpg', '_cropped.jpg'), cropped)
        print imagePath.replace('.jpg', '_cropped.jpg')

profiles/IsobelDArcy_cropped.jpg
profiles/DavidEgan_cropped.jpg
profiles/davidbabby_cropped.jpg
profiles/margaret_bridge_cropped.jpg
profiles/larettismo_cropped.jpg
profiles/Chealy__cropped.jpg
profiles/donalwallace_cropped.jpg
profiles/Carlucianomusic_cropped.jpg
profiles/amatflute_cropped.jpg
profiles/barriefromcork_cropped.jpg
profiles/joannapedrina_cropped.jpg
profiles/alispagnola_cropped.jpg
profiles/PeterTwomey_cropped.jpg
profiles/Bellaknit_cropped.jpg
profiles/dermotcasey_cropped.jpg
profiles/gavreilly_cropped.jpg
profiles/cillianmurphy_cropped.jpg
profiles/Caraquest_cropped.jpg
profiles/FiachraKinder_cropped.jpg
profiles/kencurtin_cropped.jpg
profiles/Ronan_Costello_cropped.jpg
profiles/GavinMacDermott_cropped.jpg
profiles/krisshaffer_cropped.jpg
profiles/mwrcomposer_cropped.jpg
profiles/Caitlin_Sherry_cropped.jpg
profiles/FinbarrBegley_cropped.jpg
profiles/matt_cello_cropped.jpg
profiles/DominicaLucy_cropped.jpg
profiles/oconnellhugh_cropped.jpg
profiles/wmaso_cropped.jpg
pro

In [ ]:
# At this point I move to the docker image and ran code like this:

import os
import numpy as np
import glob
# caffe_root = './caffe-master/' 
import sys
# sys.path.insert(0, caffe_root + 'python')
import caffe


mean_filename='./mean.binaryproto'
proto_data = open(mean_filename, "rb").read()
a = caffe.io.caffe_pb2.BlobProto.FromString(proto_data)
mean  = caffe.io.blobproto_to_array(a)[0]



age_net_pretrained='./age_net.caffemodel'
age_net_model_file='./deploy_age.prototxt'
age_net = caffe.Classifier(age_net_model_file, age_net_pretrained,
                       mean=mean.mean(1).mean(1),
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

gender_net_pretrained='./gender_net.caffemodel'
gender_net_model_file='./deploy_gender.prototxt'
gender_net = caffe.Classifier(gender_net_model_file, gender_net_pretrained,
                       mean=mean.mean(1).mean(1),
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

age_list=['(0, 2)','(4, 6)','(8, 12)','(15, 20)','(25, 32)','(38, 43)','(48, 53)','(60, 100)']
gender_list=['Male','Female']


image_paths= [fp for fp in glob.glob('profiles/*cropped.jpg')]

for image_path in image_paths:
  input_image = caffe.io.load_image(image_path)
  a_prediction = age_net.predict([input_image]) 
  g_prediction = gender_net.predict([input_image]) 
  print "{};{};{}".format(image_path, age_list[a_prediction[0].argmax()],  gender_list[g_prediction[0].argmax()])

In [33]:
classifier_output = """profiles/stephenoleary_cropped.jpg;(4, 6);Male
profiles/CarroljkJoe_cropped.jpg;(25, 32);Female
profiles/Taylorma91_cropped.jpg;(8, 12);Male
profiles/joewalnes_cropped.jpg;(25, 32);Male
profiles/krispharper_cropped.jpg;(15, 20);Female
profiles/NikolaiTrigoub_cropped.jpg;(25, 32);Male
profiles/ISayJK_cropped.jpg;(0, 2);Male
profiles/Mark_Sheridan_cropped.jpg;(4, 6);Female
profiles/FintanMallory_cropped.jpg;(38, 43);Female
profiles/rpblake_cropped.jpg;(25, 32);Male
profiles/matt_cello_cropped.jpg;(4, 6);Female
profiles/oconnellhugh_cropped.jpg;(8, 12);Male
profiles/dermotcasey_cropped.jpg;(4, 6);Male
profiles/Ronan_Costello_cropped.jpg;(25, 32);Male
profiles/Carlucianomusic_cropped.jpg;(25, 32);Male
profiles/amatflute_cropped.jpg;(25, 32);Female
profiles/PeterTwomey_cropped.jpg;(8, 12);Male
profiles/saratrademark_cropped.jpg;(25, 32);Female
profiles/Little_Cannon_cropped.jpg;(25, 32);Female
profiles/Caraquest_cropped.jpg;(4, 6);Female
profiles/paulmwatson_cropped.jpg;(25, 32);Male
profiles/CharleneJessop_cropped.jpg;(4, 6);Male
profiles/McManus90_cropped.jpg;(25, 32);Male
profiles/Colm_Keaveney_cropped.jpg;(25, 32);Male
profiles/cillianmurphy_cropped.jpg;(25, 32);Male
profiles/benwbrum_cropped.jpg;(25, 32);Male
profiles/alispagnola_cropped.jpg;(25, 32);Female
profiles/KelcyBoles_cropped.jpg;(25, 32);Female
profiles/DanyGshi_cropped.jpg;(8, 12);Male
profiles/DavidEgan_cropped.jpg;(25, 32);Male
profiles/margaret_bridge_cropped.jpg;(25, 32);Female
profiles/garethgregan_cropped.jpg;(25, 32);Male
profiles/awymest_cropped.jpg;(25, 32);Female
profiles/pppaattrriicckk_cropped.jpg;(25, 32);Male
profiles/hugh_odwyer_cropped.jpg;(38, 43);Male
profiles/MemesDev_cropped.jpg;(25, 32);Male
profiles/Tracy_Jentzsch_cropped.jpg;(25, 32);Female
profiles/IsobelDArcy_cropped.jpg;(0, 2);Male
profiles/joannapedrina_cropped.jpg;(25, 32);Male
profiles/Sheeeela_cropped.jpg;(38, 43);Female
profiles/Producers_IRL_cropped.jpg;(25, 32);Male
profiles/sallyhayd_cropped.jpg;(38, 43);Female
profiles/colettebrowne_cropped.jpg;(38, 43);Female
profiles/jennkirby1_cropped.jpg;(38, 43);Male
profiles/sendboyle_cropped.jpg;(25, 32);Male
profiles/enkelleh_cropped.jpg;(25, 32);Male
profiles/saeedjabbar_cropped.jpg;(25, 32);Female
profiles/davidbabby_cropped.jpg;(8, 12);Male
profiles/annaclock_cropped.jpg;(4, 6);Female
profiles/FiachraKinder_cropped.jpg;(25, 32);Male
profiles/Al_toMyFriends_cropped.jpg;(0, 2);Male
profiles/thereillywriter_cropped.jpg;(25, 32);Male
profiles/Chealy__cropped.jpg;(4, 6);Female
profiles/gavreilly_cropped.jpg;(25, 32);Female
profiles/krisshaffer_cropped.jpg;(8, 12);Male
profiles/RaymondTedders_cropped.jpg;(25, 32);Male
profiles/PohnJrafisko_cropped.jpg;(4, 6);Female
profiles/AislingCahill_cropped.jpg;(25, 32);Male
profiles/GTMFarrell_cropped.jpg;(8, 12);Female
profiles/mwrcomposer_cropped.jpg;(25, 32);Male
profiles/Hozier_cropped.jpg;(25, 32);Female
profiles/blagbtweets_cropped.jpg;(25, 32);Male
profiles/feelvall894_cropped.jpg;(25, 32);Female
profiles/emmaormusic_cropped.jpg;(25, 32);Female
profiles/glen_ster_cropped.jpg;(4, 6);Male
profiles/Bellaknit_cropped.jpg;(25, 32);Female
profiles/Jane_Deasy_cropped.jpg;(25, 32);Female
profiles/donalwallace_cropped.jpg;(25, 32);Male
profiles/larettismo_cropped.jpg;(15, 20);Male
profiles/FinbarrBegley_cropped.jpg;(25, 32);Female
profiles/GavinMacDermott_cropped.jpg;(48, 53);Male
profiles/Caitlin_Sherry_cropped.jpg;(25, 32);Female
profiles/kencurtin_cropped.jpg;(25, 32);Male
profiles/jeannedesutun_cropped.jpg;(25, 32);Female
profiles/niamhnicu_cropped.jpg;(25, 32);Female
profiles/wmaso_cropped.jpg;(25, 32);Male
profiles/ThisFashions_cropped.jpg;(25, 32);Male
profiles/meganlavengood_cropped.jpg;(38, 43);Female
profiles/NaoiseDolan_cropped.jpg;(25, 32);Female
profiles/micewhel_cropped.jpg;(25, 32);Male
profiles/barriefromcork_cropped.jpg;(8, 12);Male
profiles/LILYmk_cropped.jpg;(8, 12);Female
profiles/HigginsDav_cropped.jpg;(4, 6);Male
profiles/MeganCanavan1_cropped.jpg;(4, 6);Male
profiles/DominicaLucy_cropped.jpg;(15, 20);Male
profiles/annieswafford_cropped.jpg;(15, 20);Male
profiles/nyscandia_cropped.jpg;(25, 32);Female
profiles/SenDoherty_cropped.jpg;(0, 2);Male
profiles/ValComedy_cropped.jpg;(48, 53);Male
profiles/KateATipp_cropped.jpg;(25, 32);Female
profiles/Harmonica26_cropped.jpg;(25, 32);Female
profiles/DonnellyStephen_cropped.jpg;(38, 43);Female
profiles/rianderrig_cropped.jpg;(38, 43);Female
profiles/PatrickPio1_cropped.jpg;(15, 20);Female
profiles/aifricdennison_cropped.jpg;(25, 32);Female
profiles/foreveralowen_cropped.jpg;(25, 32);Male"""

In [45]:
output = [x.split(';') for x in classifier_output.split('\n')]

In [46]:
for o in output:
    o[0] = o[0][9:].replace('_cropped.jpg', '')

In [48]:
# Twitter handle, age range, gender

output

[['stephenoleary', '(4, 6)', 'Male'],
 ['CarroljkJoe', '(25, 32)', 'Female'],
 ['Taylorma91', '(8, 12)', 'Male'],
 ['joewalnes', '(25, 32)', 'Male'],
 ['krispharper', '(15, 20)', 'Female'],
 ['NikolaiTrigoub', '(25, 32)', 'Male'],
 ['ISayJK', '(0, 2)', 'Male'],
 ['Mark_Sheridan', '(4, 6)', 'Female'],
 ['FintanMallory', '(38, 43)', 'Female'],
 ['rpblake', '(25, 32)', 'Male'],
 ['matt_cello', '(4, 6)', 'Female'],
 ['oconnellhugh', '(8, 12)', 'Male'],
 ['dermotcasey', '(4, 6)', 'Male'],
 ['Ronan_Costello', '(25, 32)', 'Male'],
 ['Carlucianomusic', '(25, 32)', 'Male'],
 ['amatflute', '(25, 32)', 'Female'],
 ['PeterTwomey', '(8, 12)', 'Male'],
 ['saratrademark', '(25, 32)', 'Female'],
 ['Little_Cannon', '(25, 32)', 'Female'],
 ['Caraquest', '(4, 6)', 'Female'],
 ['paulmwatson', '(25, 32)', 'Male'],
 ['CharleneJessop', '(4, 6)', 'Male'],
 ['McManus90', '(25, 32)', 'Male'],
 ['Colm_Keaveney', '(25, 32)', 'Male'],
 ['cillianmurphy', '(25, 32)', 'Male'],
 ['benwbrum', '(25, 32)', 'Male'],
 ['al